<a href="https://colab.research.google.com/github/ansh-pachauri/GenAI-project-on-collab/blob/main/titanic_using_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv/train.csv


In [5]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('train.csv/train.csv')

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [10]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42
    )

In [13]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [16]:
y_train.sample(5)

,Survived
723,0
310,1
85,1
562,0
737,1


In [17]:
# transformers create

In [18]:
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(),[2]),
    ('inpute_embarked', SimpleImputer(strategy='most_frequent'),[6])
], remainder='passthrough'
)

In [40]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[1,6])
], remainder='passthrough'
)

In [47]:
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,8))
])

In [48]:
trf4 = SelectKBest(score_func=chi2, k=5)

In [49]:
trf5 = DecisionTreeClassifier()

In [50]:
# Creating Pipeline

In [51]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),('trf4',trf4),
    ('trf5',trf5),
])

In [52]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('inpute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('trf4',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x78da98a10cc0>)),
                ('trf5', DecisionTreeClassifier())])

In [38]:
from sklearn import set_config
set_config(display='diagram')

In [53]:
y_predict = pipe.predict(X_test)

In [55]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.6256983240223464

In [60]:
# Cross Validation using pipeline
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

np.float64(0.6391214419383433)